# Project Data Visualisation

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import folium
import math
from ipywidgets import interact, IntSlider, Layout
import json
import requests
from pyspark.sql import functions as f
from shapely.geometry import shape
from geopy.distance import geodesic
import numpy as np
spark = SparkSession.builder.master("local[*]").appName("notebook").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(spark)

C:\Users\rstep\AppData\Roaming\Python\Python312\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
### ZIEKENHUIZEN DATA ###

# Data inlezen
ziekenhuizen = spark.read.format('csv').option('header', 'true').load('./data/ziekenhuizen.csv')

# Laden van de eerste 5 rijen om een idee te krijgen van de data
ziekenhuizen.show(5)

# Relevante kolommen filteren
ziekenhuizen_filtered = ziekenhuizen.select(
    'X', 'Y', 'NAAM', 
    'LABEL', 'STRAATNAAM', 'HUISNUMMER', 
    'POSTCODE', 'TOTALE_CAP', 
    'TELEFOON', 'EMAIL', 'WIJK')

min_lon, max_lon = 4.35, 4.45
min_lat, max_lat = 51.10, 51.40

# Functie om Mercator naar LonLat om te zetten
def mercator_to_lonlat(x, y):
    R = 6378137.0
    lon = (x / R) * (180.0 / math.pi)
    lat = (math.pi/2.0 - 2.0 * math.atan(math.exp(-y / R))) * (180.0 / math.pi)
    return lon, lat

# Ziekenhuis markers aanmaken ({(lat, lon): popup_html})
ziekenhuis_markers = {}
for row in ziekenhuizen_filtered.collect():
    lon, lat = mercator_to_lonlat(float(row['X']), float(row['Y']))
    # Popup HTML
    html = f"""
    <div>
      <strong>{row['LABEL'] or row['NAAM'] or ''}</strong><br/>
      {'<strong>Adres:</strong> ' + (row['STRAATNAAM'] + ' ' or '') + (row['HUISNUMMER'] or '') +'<br/>' if row['STRAATNAAM'] and row['HUISNUMMER'] else ''}
      {'<strong>Postcode:</strong> ' + (row['POSTCODE'] or '') + '<br/>' if row['POSTCODE'] else ''}
      {'<strong>Totale capaciteit:</strong> ' + (row['TOTALE_CAP'] or '') + '<br/>' if row['TOTALE_CAP'] and row['TOTALE_CAP'].strip() else ''}
      {'<strong>Telefoon:</strong> ' + (row['TELEFOON'] or '') + '<br/>' if row['TELEFOON'] and row['TELEFOON'].strip() else ''}
      {'<strong>Email:</strong> ' + (row['EMAIL'] or '') + '<br/>' if row['EMAIL'] else ''}
    </div>
    """
    ziekenhuis_markers[(lat, lon)] = folium.Popup(folium.Html(html, script=True), max_width=300)


+----------------+----------------+--------+---------+--------------------+--------------------+--------------------+----------+--------+----------+-------------------+---------------+--------------------+----------+------+---------+--------------------+----------+--------+------------+--------------+----------+-----+
|               X|               Y|OBJECTID|BEHEERDER|                NAAM|               LABEL|          STRAATNAAM|HUISNUMMER|POSTCODE|ID_COMPLEX|               TYPE|JURIDISCHE_VORM|                WIJK|TOTALE_CAP|NUMMER|ERKENNING|                LINK|INFORMATIE|GEMEENTE|    TELEFOON|         EMAIL|  DISTRICT|SPOED|
+----------------+----------------+--------+---------+--------------------+--------------------+--------------------+----------+--------+----------+-------------------+---------------+--------------------+----------+------+---------+--------------------+----------+--------+------------+--------------+----------+-----+
|492279.700722991|6653768.33545457|    1

In [3]:
### GEOJSON NAMEN OMZETTEN
url = 'https://geodata.antwerpen.be/arcgissql/rest/services/P_Portal/portal_publiek2/MapServer/97/query?outFields=*&where=1%3D1&f=geojson'

def safe_get_properties(feature):
    properties_data = feature.get('properties')
    if isinstance(properties_data, str):
        try:
            return json.loads(properties_data)
        except json.JSONDecodeError:
            return {}
    return properties_data

try:
    response = requests.get(url)
    response.raise_for_status()
    geo_json_data = response.json()
except requests.exceptions.RequestException as e:
    print(f"Fout bij het laden van GeoJSON URL: {e}. Gebruik lege data.")
    geo_json_data = {"type": "FeatureCollection", "features": []}

# Wijken namen mapping om later in de code te gebruiken
wijknamen = {
    "Antwerpen - Amandus & Atheneum": "Amandus - Atheneum",
    "Antwerpen - Brederode": "Brederode",
    "Antwerpen - Centraal station": "Centraal Station",
    "Antwerpen - Dam": "Dam",
    "Antwerpen - Eilandje": "Eilandje",
    "Antwerpen - Haringrode": "Haringrode",
    "Antwerpen - Harmonie": "Harmonie",
    "Antwerpen - Haven": "Haven Antwerpen",
    "Antwerpen - Historisch centrum": "Historisch Centrum",
    "Antwerpen - Kiel": "Kiel",
    "Antwerpen - Linkeroever": "Linkeroever",
    "Antwerpen - Luchtbal": "Luchtbal",
    "Antwerpen - Markgrave": "Markgrave",
    "Antwerpen - Middelheim": "Middelheim",
    "Antwerpen - Nieuw-zuid": "Nieuw - Zuid",
    "Antwerpen - Petroleum-zuid": "Petroleum - Zuid",
    "Antwerpen - Sint-Andries": "Sint-Andries",
    "Antwerpen - Stadspark": "Stadspark",
    "Antwerpen - Stuivenberg": "Stuivenberg",
    "Antwerpen - Tentoonstellingswijk": "Tentoonstellingswijk",
    "Antwerpen - Theaterbuurt & Meir": "Theaterbuurt-Meir",
    "Antwerpen - Universiteitsbuurt": "Universiteitsbuurt",
    "Antwerpen - Zuid": "Zuid",
    "Antwerpen - Zurenborg": "Zurenborg",
    "Berchem - Groenenhoek": "Groenenhoek",
    "Berchem - Nieuw-Kwartier (oost)": "Nieuw - Kwartier Oost",
    "Berchem - Nieuw-Kwartier (west)": "Nieuw - Kwartier West",
    "Berchem - Oud-Berchem": "Oud - Berchem",
    "Bezali - Haven": "Haven Bezali",
    "Bezali - Lillo": "Lillo",
    "Bezali - Polder": "Polder",
    "Borgerhout - Extra muros": "Borgerhout - Extra Muros",
    "Borgerhout - Intra muros (noord)": "Borgerhout Intra Muros Noord",
    "Borgerhout - Intra muros (zuid)": "Borgerhout Intra Muros Zuid",
    "Deurne - Noord": "Deurne - Noord",
    "Deurne - Kuininge & Bremweide": "Kruininge - Bremweide",
    "Deurne - Dorp & Gallifort": "Deurne Dorp - Gallifort",
    "Deurne - Oost": "Deurne - Oost",
    "Deurne - Zuidwest": "Deurne - Zuidwest",
    "Deurne - Zuidoost": "Deurne - Zuidoost",
    "Deurne - Vlieghaven": "Deurne Vlieghaven",
    "Deurne - Rivierenhof": "Rivierenhof",
    "Ekeren - Donk": "Donk",
    "Ekeren - Centrum": "Ekeren Centrum",
    "Ekeren - Leugenberg": "Leugenberg",
    "Ekeren - Mariaburg": "Mariaburg",
    "Ekeren - Schoonbroek & Rozemaai": "Schoonbroek-Rozemaai",
    "Ekeren - Muisbroek & Bospolder": "Muisbroek-Bospolder",
    "Hoboken - Centrum": "Hoboken - Centrum",
    "Hoboken - Noord": "Hoboken - Noord",
    "Hoboken - West": "Hoboken - West",
    "Hoboken - Zuidoost": "Hoboken - Zuidoost",
    "Merksem - Lambrechtshoeken": "Lambrechtshoeken",
    "Merksem - Heide": "Merksem - Heide",
    "Merksem - Nieuwdreef": "Nieuwdreef",
    "Merksem - Oud-Merksem": "Oud - Merksem",
    "Merksem - Tuinwijk": "Tuinwijk",
    "Schelde": "Schelde", 
    "Wilrijk - Hoogte": "Hoogte",
    "Wilrijk - Oosterveld & Elsdonk": "Oosterveld - Elsdonk",
    "Wilrijk - Valaar": "Valaar",
    "Wilrijk - Centrum": "Wilrijk Centrum",
    "Wilrijk - Koornbloem": "Koornbloem",
    "Wilrijk - Neerland": "Neerland",
    "Borsbeek - Groen": "Borsbeek-Groen",
    "Borsbeek - Oost": "Borsbeek-Oost",
    "Borsbeek - West": "Borsbeek-West"
}

for feature in geo_json_data['features']:
    properties = safe_get_properties(feature)    
    geo_json_raw_name = properties.get('wijknaam')
    if geo_json_raw_name:
        correcte_sleutel = wijknamen.get(geo_json_raw_name, geo_json_raw_name)         
        properties['wijknaam'] = correcte_sleutel
        feature['properties'] = properties
        #print(f"Omgezet: {geo_json_raw_name} -> {correcte_sleutel}")

In [4]:
### AFSTAND ZIEKENHUIZEN EN WIJKEN ###

# Ziekenhuis coördinaten
ziekenhuizen_locaties = []
for zh in ziekenhuizen_filtered.collect():
    lon, lat = mercator_to_lonlat(float(zh['X']), float(zh['Y']))
    ziekenhuizen_locaties.append((lat, lon))

# Loop door elke wijk in de GeoJSON data
for feature in geo_json_data['features']:
    geometry = feature.get('geometry')

    # Veilig ophalen van properties
    properties = feature.get('properties', {})
    if geometry and geometry.get('coordinates'):
        try:
            # Middelpunt van de wijk berekenen
            wijk_poly = shape(geometry)
            centroid = wijk_poly.centroid
            
            centroid_lon = centroid.x
            centroid_lat = centroid.y
            
            wijk_coord = (centroid_lat, centroid_lon)
            
            #Afstanden naar alle ziekenhuizen berekenen
            afstanden = []
            for ziekenhuis_coord in ziekenhuizen_locaties:                
                afstand = geodesic(wijk_coord, ziekenhuis_coord).km
                afstanden.append(afstand)
            min_afstand_km = np.min(afstanden)
            
            # Voeg nieuwe data toe aan GEOJSON
            properties['centroid_lat'] = centroid_lat
            properties['centroid_lon'] = centroid_lon
            properties['min_afstand_km'] = round(min_afstand_km, 2)
        # Error handling  
        except Exception as e:
            print(f"Fout bij verwerken wijk {properties.get('wijknaam', 'onbekend')}: {e}")
            properties['min_afstand_km'] = None
    else:
        properties['min_afstand_km'] = None
    # Terug schrijven van properties
    feature['properties'] = properties

# Normaliseer afstand voor kleurgebruik
max_afstand = 10 
def normalize_distance(afstand):
    if afstand is None or afstand < 0:
        return 0
    afstand = min(afstand, max_afstand)
    normalized_value = (afstand / max_afstand) * 255
    return int(normalized_value)

In [5]:
### SOCIO-ECONOMISCHE DATA ###
# Data inlezen
socio_data = spark.read.format('csv').option('header', 'true').option('delimiter', ';').load('./data/wijkenranking.csv')

# Data schoonmaken
score_col = "Wijkenranking: totaalscore sociaal-economische situatie  (0 tot 100)|2025"
socio_data_cleaned = (
    socio_data.withColumn(score_col, f.when((f.col(score_col).isNull()) | (f.col(score_col) == ""),f.lit("0")).otherwise(f.col(score_col)))
    .withColumn(score_col, f.regexp_replace(f.col(score_col), r"\s+", ""))
    .withColumn(score_col, f.col(score_col).cast("float"))
)
socio_data = socio_data_cleaned

# Omzetten naar dictionary
socio_rows= {
    row["Wijken"]: row["Wijkenranking: totaalscore sociaal-economische situatie  (0 tot 100)|2025"]
    for row in socio_data.collect()
}

# Normaliseer score voor kleurgebruik
def normalize_socio(value):
    if value is None:
        return 0
    return int((value / 100) * 255)


In [6]:
### LEEFTIJD PER WIJK MAP ###
# Data inlezen
leeftijd_wijk = spark.read.format('csv').option('header', 'true').option('delimiter', ';').load('./data/leeftijd_per_wijk.csv')

# Laden van de eerste 5 rijen om een idee te krijgen van de data
leeftijd_wijk.show(5)

# Kolomnamen hernoemen voor meer duidelijkheid
leeftijd_wijk = leeftijd_wijk \
    .withColumnRenamed("Wijken", "wijknaam") \
    .withColumnRenamed("Aantal 0 tem 9-jarigen|2025", "0-9") \
    .withColumnRenamed("Aantal 10 tem 19-jarigen|2025", "10-19") \
    .withColumnRenamed("Aantal 20 tem 29-jarigen|2025", "20-29") \
    .withColumnRenamed("Aantal 30 tem 39-jarigen|2025", "30-39") \
    .withColumnRenamed("Aantal 40 tem 49-jarigen|2025", "40-49") \
    .withColumnRenamed("Aantal 50 tem 59-jarigen|2025", "50-59") \
    .withColumnRenamed("Aantal 60 tem 69-jarigen|2025", "60-69") \
    .withColumnRenamed("Aantal 70 tem 79-jarigen|2025", "70-79") \
    .withColumnRenamed("Aantal 80 tem 89-jarigen|2025", "80-89") \
    .withColumnRenamed("Aantal 90+'ers|2025", "90plus")

# Data omzetten naar integer en lege waarden afhandelen
leeftijden = [
    "0-9", "10-19", "20-29", "30-39", "40-49", 
    "50-59", "60-69", "70-79", "80-89", "90plus"
]

# Data schoonmaken
for c in leeftijden:
    leeftijd_wijk = (leeftijd_wijk
        .withColumn(c, f.when(f.col(c) == ".", 0).otherwise(f.col(c)))
        .withColumn(c, f.regexp_replace(f.col(c), r"\s+", ""))
        .withColumn(c, f.col(c).cast("int"))
    )

# Omzetten naar dictionary
inwoners_rows = leeftijd_wijk.select("wijknaam", *leeftijden).collect()
inwoners_data = {
    row["wijknaam"]: {age_col: row[age_col] for age_col in leeftijden}
    for row in inwoners_rows
}

# Bereken maximum aantal inwoners per leeftijdsgroep
max_inwoners = {c: leeftijd_wijk.select(f.max(c)).first()[0] for c in leeftijden}
#print(max_inwoners)
absolute_max = max(max_inwoners.values())
#print(absolute_max)

# Normaliseer aantal inwoners voor kleurgebruik
def normalize_inwoners(value):
    if value is None:
        return 0
    return int((value / absolute_max) * 255)

# Slider maken voor leeftijdsgroepen
age_slider = IntSlider(
    value=0,
    min=0,
    max=(len(leeftijden) - 1) * 10,
    step=10,
    description='Leeftijd:',
    continuous_update=False,
    layout=Layout(width='500px')
)

popup = folium.GeoJsonPopup(fields=["wijknaam"], aliases=["Wijk naam:"])

@interact(age_group=age_slider)
def update_map(age_group):
    m = folium.Map(
        max_bounds=True,
        location=(51.23016715, 4.4161294643975015),
        zoom_start=11,
        min_zoom=10,
        max_lat=max_lat,
        min_lat=min_lat,
        min_lon=min_lon,
        max_lon=max_lon,
    )    
    folium.TileLayer('Cartodb Positron').add_to(m)
    
    for location, marker in ziekenhuis_markers.items():
         folium.Marker(
            location=location,
            popup=marker,
            icon=folium.Icon(color='red', icon='plus-sign')
        ).add_to(m)
    
    # Gewichten voor elke kleurcomponent
    weight_inwoners = 0.5
    weight_socio = 0.3
    weight_afstand = 0.2

    selected_age = leeftijden[age_group//10]
    def style_function(feature):
        wijk = feature['properties']['wijknaam']

        # Haal aantal inwoners uit wijk
        if wijk in inwoners_data:
            wijk_data = inwoners_data[wijk]
            inwoners = wijk_data.get(selected_age)
        else:
            inwoners = 0 
        
        # Haal socio-economische score uit wijk
        if wijk in socio_rows:
            socio_score = socio_rows.get(wijk)
        else:
            socio_score = 0 

        # Haal berekende afstand uit wijk
        afstand_wijk = feature['properties'].get('min_afstand_km')

        # Bepaal kleur afhankelijk van:
        # afstand tot dichtstbijzijnde ziekenhuis
        color_afstand = normalize_distance(afstand_wijk)
        # aantal inwoners in de geselecteerde leeftijdsgroep
        color_inwoners = normalize_inwoners(inwoners)
        # socio-economische situatie
        color_socio = normalize_socio(socio_score)

        # Gewogen gemiddelde van alle kleuren nemen
        total_color = int((color_inwoners * weight_inwoners) + (color_socio * weight_socio) + (color_afstand * weight_afstand))
        # Debug output
        #print("Wijk:", wijk,"inwoners:", inwoners, "score:", socio_score, "kleur:", total_color, "afstand:", afstand_wijk)
        red_component = total_color
        green_component = 255 - total_color
        if total_color == 0:
            color = 'grey'
        else:
            color = f'#{red_component:02x}{green_component:02x}00' 

        return {
            "fillColor": color,
            "color": "black",
            "weight": 1,
            "fillOpacity": 0.6
        }

    # Voeg wijken toe met kleur op basis van geselecteerde leeftijdsgroep
    folium.GeoJson(
    geo_json_data,
    name="Wijk grenzen",
    style_function=style_function,
    highlight_function=lambda feature: {
        "fillColor": "grey",
        "color": "black",
        "weight": 3,
        "fillOpacity": 0.6,
    },
    popup=popup,
    popup_keep_highlighted=True,
    ).add_to(m)
    
    folium.LayerControl().add_to(m)
    display(m)
    m.save('index.html')

+------------------+---------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-------------------+
|            Wijken|Aantal 0 tem 9-jarigen|2025|Aantal 10 tem 19-jarigen|2025|Aantal 20 tem 29-jarigen|2025|Aantal 30 tem 39-jarigen|2025|Aantal 40 tem 49-jarigen|2025|Aantal 50 tem 59-jarigen|2025|Aantal 60 tem 69-jarigen|2025|Aantal 70 tem 79-jarigen|2025|Aantal 80 tem 89-jarigen|2025|Aantal 90+'ers|2025|
+------------------+---------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-------------------+
|Amandus - Atheneum|                       2112|                         

interactive(children=(IntSlider(value=0, continuous_update=False, description='Leeftijd:', layout=Layout(width…